# Imports

In [15]:
from __future__ import division, print_function, absolute_import, unicode_literals

import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pk
from time import time
from typing import Text
import tensorflow as tf

import sys
sys.path.insert(0, '../src/')
import utils
import text_processor
import broadcast_network_extraction


In [16]:
net_extractor = broadcast_network_extraction.NetworkExtraction()
content_fixer = text_processor.FormalEnglishTranslator('../bagofwords/slang.txt')

In [17]:
directory = '/Users/koasato/Documents/koa/College/UCSB/2019-2020/Research/Jeopardy/'
data = None
try:
    data = utils.load_it(directory + 'Teams_logs.pk')
except:
    print("done")
    


print(len(data))

49


In [18]:
combined_logs = {}
for team_id, team_log in data.items():
    print("Processing team", team_id, '...')
    this_team_nets = []
    this_team_number_of_networks = min(
        len(team_log.messages) // 5,
        len(team_log.member_influences))
    all_messages_before_appraisal_reports = []
    for i in range(this_team_number_of_networks):
        all_messages_before_appraisal_reports.append(
            pd.concat(
                [team_log.messages[i] for i in np.arange(i * 5, i * 5 + 5)]))
    if len(all_messages_before_appraisal_reports) > 0:
        combined_logs[team_id] = all_messages_before_appraisal_reports
    else:
        print('Team', team_id, 'does not have enough logs.')


Processing team 7 ...
Processing team 10 ...
Processing team 11 ...
Processing team 12 ...
Processing team 13 ...
Processing team 14 ...
Processing team 15 ...
Processing team 16 ...
Processing team 17 ...
Processing team 19 ...
Processing team 20 ...
Processing team 21 ...
Processing team 22 ...
Processing team 23 ...
Processing team 27 ...
Processing team 28 ...
Processing team 30 ...
Processing team 31 ...
Processing team 32 ...
Processing team 33 ...
Processing team 34 ...
Processing team 35 ...
Team 35 does not have enough logs.
Processing team 36 ...
Processing team 37 ...
Processing team 38 ...
Processing team 39 ...
Processing team 40 ...
Processing team 41 ...
Processing team 42 ...
Processing team 43 ...
Processing team 44 ...
Processing team 45 ...
Processing team 46 ...
Processing team 47 ...
Processing team 48 ...
Processing team 49 ...
Processing team 70 ...
Processing team 71 ...
Processing team 72 ...
Team 72 does not have enough logs.
Processing team 73 ...
Team 73 doe

In [29]:
for b in range(2,5):
    for e in range(6, 12):
        if b >= e:
            continue
        time_window = [b, e]
        print("time window = ", time_window)
        for team_id, all_messages_before_appraisal_reports in combined_logs.items():
            print("Processing team", team_id, '...')
            this_team_nets = []
            for index, all_messages_before_appraisal_report in enumerate(all_messages_before_appraisal_reports):
                reply_duration_net = net_extractor.extract_network_from_broadcast(            
                    communication_data=all_messages_before_appraisal_report,
                    time_window=time_window,
                    weight_type=broadcast_network_extraction.WeightType.REPLY_DURATION,
                    aggregation_type=broadcast_network_extraction.AggregationType.SUM,
                    gamma=0.15,
                    node_list=data[team_id].members)
                influence_matrix = np.matrix(data[team_id].member_influences[index])
                to_be_predicted_influence_matrix = utils.shuffle_matrix_in_given_order(
                        matrix=influence_matrix,
                        order=np.argsort(team_log.members)) / 100
                # Use reply_duration_net to predict to_be_predicted_influence_matrix
                
                # DEBUG PRINTS
                print("reply_net = ", reply_duration_net)
                print("inf_mat = ", to_be_predicted_influence_matrix)
                sub = (reply_duration_net - to_be_predicted_influence_matrix)
                mse = (np.square(reply_duration_net - to_be_predicted_influence_matrix)).mean
                print(sub)
                


time window =  [2, 6]
Processing team 7 ...
reply_net =  
inf_mat =  [[0.97 0.01 0.01 0.01]
 [0.25 0.25 0.25 0.25]
 [0.2  0.4  0.   0.4 ]
 [0.   0.   1.   0.  ]]
[[16.03 19.99 18.99 17.99]
 [16.75 19.75 18.75 17.75]
 [16.8  19.6  19.   17.6 ]
 [17.   20.   18.   18.  ]]
reply_net =  
inf_mat =  [[0.97 0.01 0.01 0.01]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.7  0.05 0.2  0.05]]
[[16.03 19.99 18.99 17.99]
 [16.75 19.75 18.75 17.75]
 [16.75 19.75 18.75 17.75]
 [16.3  19.95 18.8  17.95]]
reply_net =  
inf_mat =  [[0.97 0.01 0.01 0.01]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.6  0.1  0.2  0.1 ]]
[[16.03 19.99 18.99 17.99]
 [16.75 19.75 18.75 17.75]
 [16.75 19.75 18.75 17.75]
 [16.4  19.9  18.8  17.9 ]]
reply_net =  
inf_mat =  [[0.96 0.   0.02 0.02]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.3  0.2  0.3  0.2 ]]
[[16.04 20.   18.98 17.98]
 [16.75 19.75 18.75 17.75]
 [16.75 19.75 18.75 17.75]
 [16.7  19.8  18.7  17.8 ]]
reply_net =  
inf_mat =  [[0.95 0.01 0.02 0.02]


reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]]
reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]]
reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]]
reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]
 [32.75 33.75 34.75 35.75]]
reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.2  0.4  0.2  0.2 ]
 [0.25 0.25 0.25 0.25

reply_net =  
inf_mat =  [[0.1  0.2  0.4  0.3 ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.2  0.3  0.2  0.3 ]]
[[74.9  76.8  73.6  75.7 ]
 [74.75 76.75 73.75 75.75]
 [74.75 76.75 73.75 75.75]
 [74.8  76.7  73.8  75.7 ]]
Processing team 21 ...
reply_net =  
inf_mat =  [[0.25 0.25 0.3  0.2 ]
 [0.37 0.14 0.33 0.16]
 [0.25 0.25 0.25 0.25]
 [0.15 0.45 0.25 0.15]]
[[80.75 78.75 77.7  79.8 ]
 [80.63 78.86 77.67 79.84]
 [80.75 78.75 77.75 79.75]
 [80.85 78.55 77.75 79.85]]
reply_net =  
inf_mat =  [[0.15 0.35 0.3  0.2 ]
 [0.4  0.14 0.3  0.16]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[80.85 78.65 77.7  79.8 ]
 [80.6  78.86 77.7  79.84]
 [80.75 78.75 77.75 79.75]
 [80.75 78.75 77.75 79.75]]
reply_net =  
inf_mat =  [[0.2  0.3  0.25 0.25]
 [0.27 0.2  0.26 0.27]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
[[80.8  78.7  77.75 79.75]
 [80.73 78.8  77.74 79.73]
 [80.75 78.75 77.75 79.75]
 [80.75 78.75 77.75 79.75]]
reply_net =  
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.25 0.2  0.22 0.25

KeyboardInterrupt: 